# Evaluate a Siamese model: Ungraded Lecture Notebook

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.math as math
import tensorflow.linalg as linalg

2024-02-02 03:42:07.630502: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 03:42:07.762478: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 03:42:07.763782: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 03:42:08.673476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Inspecting the necessary elements

In this lecture notebook you will learn how to evaluate a Siamese model using the accuracy metric. Because there are many steps before evaluating a Siamese network (as you will see in this week's assignment) the necessary elements and variables are replicated here using real data from the assignment:

   - `q1`: vector with dimension `(batch_size X max_length)` containing first questions to compare in the test set.
   - `q2`: vector with dimension `(batch_size X max_length)` containing second questions to compare in the test set.
   
   **Notice that for each pair of vectors within a batch $([q1_1, q1_2, q1_3, ...]$, $[q2_1, q2_2,q2_3, ...])$  $q1_i$ is associated to $q2_k$.**
        
        
   - `y_test`: 1 if  $q1_i$ and $q2_k$ are duplicates, 0 otherwise.
   
   - `v1`: output vector from the model's prediction associated with the first questions.
   - `v2`: output vector from the model's prediction associated with the second questions.

You can inspect each one of these variables by running the following cells:

In [2]:
q1 = np.load('./data/q1.npy')
print(f'q1 has shape: {q1.shape} \n\nAnd it looks like this: \n\n {q1}\n\n')

q1 has shape: (512, 64) 

And it looks like this: 

 [[ 32  38   4 ...   1   1   1]
 [ 30 156  78 ...   1   1   1]
 [ 32  38   4 ...   1   1   1]
 ...
 [ 32  33   4 ...   1   1   1]
 [ 30 156 317 ...   1   1   1]
 [ 30 156   6 ...   1   1   1]]




Notice those 1s on the right-hand side?  

Hope you remember that the value of `1` was used for padding. 

In [3]:
q2 = np.load('./data/q2.npy')
print(f'q2 has shape: {q2.shape} \n\nAnd looks like this: \n\n {q2}\n\n')

q2 has shape: (512, 64) 

And looks like this: 

 [[   30   156    78 ...     1     1     1]
 [  283   156    78 ...     1     1     1]
 [   32    38     4 ...     1     1     1]
 ...
 [   32    33     4 ...     1     1     1]
 [   30   156    78 ...     1     1     1]
 [   30   156 10596 ...     1     1     1]]




In [4]:
y_test = np.load('./data/y_test.npy')
print(f'y_test has shape: {y_test.shape} \n\nAnd looks like this: \n\n {y_test}\n\n')

y_test has shape: (512,) 

And looks like this: 

 [0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0
 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1
 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1
 1 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0
 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0
 0 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 1 1 0
 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1
 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 0 1 

In [5]:
v1 = np.load('./data/v1.npy')
print(f'v1 has shape: {v1.shape} \n\nAnd looks like this: \n\n {v1}\n\n')
v2 = np.load('./data/v2.npy')
print(f'v2 has shape: {v2.shape} \n\nAnd looks like this: \n\n {v2}\n\n')

v1 has shape: (512, 128) 

And looks like this: 

 [[ 0.01273625 -0.1496373  -0.01982759 ...  0.02205012 -0.00169148
  -0.01598107]
 [-0.05592084  0.05792497 -0.02226785 ...  0.08156938 -0.02570007
  -0.00503111]
 [ 0.05686752  0.0294889   0.04522024 ...  0.03141788 -0.08459651
  -0.00968536]
 ...
 [ 0.15115018  0.17791134  0.02200656 ... -0.00851707  0.00571415
  -0.00431194]
 [ 0.06995274  0.13110274  0.0202337  ... -0.00902792 -0.01221745
   0.00505962]
 [-0.16043712 -0.11899089 -0.15950686 ...  0.06544471 -0.01208312
  -0.01183368]]


v2 has shape: (512, 128) 

And looks like this: 

 [[ 0.07437647  0.02804951 -0.02974014 ...  0.02378932 -0.01696189
  -0.01897198]
 [ 0.03270066  0.15122835 -0.02175895 ...  0.00517202 -0.14617395
   0.00204823]
 [ 0.05635608  0.05454165  0.042222   ...  0.03831453 -0.05387777
  -0.01447786]
 ...
 [ 0.04727105 -0.06748016  0.04194937 ...  0.07600753 -0.03072828
   0.00400715]
 [ 0.00269269  0.15222628  0.01714724 ...  0.01482705 -0.0197884
   0.01389

## Calculating the accuracy

You will calculate the accuracy by iterating over the test set and checking if the model predicts right or wrong.

The first step is to set the accuracy to zero:

In [6]:
accuracy = 0

You will also need the `batch size` and the `threshold` that determines if two questions are the same or not. 

**Note :A higher threshold means that only very similar questions will be considered as the same question.**

In [7]:
batch_size = 512 # Note: The max it can be is y_test.shape[0] i.e all the samples in test data
threshold = 0.7 # You can play around with threshold and then see the change in accuracy.

In the assignment you will iterate over multiple batches of data but since this is a simplified version only one batch is provided. 

**Note: Be careful with the indices when slicing the test data in the assignment!**

The process is pretty straightforward:
   - Iterate over each one of the elements in the batch
   - Compute the cosine similarity between the predictions
       - For computing the cosine similarity, the two output vectors should have been normalized using L2 normalization meaning their magnitude will be 1. This has been taken care of by the Siamese network you will build in the assignment. Hence the cosine similarity here is just dot product between two vectors. You can check by implementing the usual cosine similarity formula and check if this holds or not.
   - Determine if this value is greater than the threshold (If it is, consider the two questions as the same and return 1 else 0)
   - Compare against the actual target and if the prediction matches, add 1 to the accuracy (increment the correct prediction counter)
   - Divide the accuracy by the number of processed elements

In [8]:
for j in range(batch_size):        # Iterate over each element in the batch
    print(v1[j].shape)
    print(v2[j].shape)
    print(v1[j])
    print(v2[j])
    d = math.reduce_sum(v1[j]*v2[j])# Compute the cosine similarity between the predictions as l2 normalized, ||v1[j]||==||v2[j]||==1 so only dot product is needed
    print('Cos Similarity ',d)
    res = d > threshold            # Determine if this value is greater than the threshold (if it is consider the two questions as the same)
    accuracy += tf.cast(y_test[j] == res, tf.int32) # Compare against the actual target and if the prediction matches, add 1 to the accuracy

accuracy = accuracy / batch_size   # Divide the accuracy by the number of processed elements

(128,)
(128,)
[ 1.27362460e-02 -1.49637297e-01 -1.98275950e-02 -1.08946666e-01
 -1.68642197e-02 -1.65867269e-01  1.07190855e-01 -4.33174968e-02
  9.07681510e-03 -5.92368562e-03 -6.78903759e-02  2.61136866e-03
  7.75974302e-04 -1.77034229e-01 -1.37154330e-02  3.75972353e-02
  4.30502789e-03 -9.24129561e-02 -1.83787912e-01 -9.10479575e-02
 -3.44949439e-02  3.88996266e-02 -1.47246629e-01  5.85732386e-02
  7.21155014e-03  1.73979648e-03  2.09133610e-01 -8.61155707e-03
 -8.30050372e-03  1.16931684e-01  1.52790904e-01 -1.83973595e-01
 -9.09670489e-04 -2.16546282e-02  6.64035976e-02  1.48809804e-02
 -1.08917868e-02 -7.68949613e-02  3.59993964e-03 -6.51998892e-02
  1.99166477e-01 -1.83426333e-03 -1.80561282e-03 -9.94487666e-03
 -8.65307748e-02 -2.01043651e-01  1.08571388e-01  3.02748079e-03
  8.81575141e-03  1.12046637e-02  1.03988880e-02 -4.71503299e-05
 -1.54345399e-02  3.81085370e-03 -1.76025648e-02 -2.37256149e-03
 -6.52001277e-02  1.45423552e-03  9.52280220e-03 -4.58056666e-03
 -1.6954909

Cos Similarity  tf.Tensor(0.898223, shape=(), dtype=float32)
(128,)
(128,)
[ 7.87233040e-02  3.76037434e-02  7.74443895e-02 -1.97058275e-01
  2.15158332e-02  9.59832221e-02  1.88869163e-01  1.29640818e-01
 -7.48610776e-03  8.80182907e-03 -2.08553374e-01  4.37426083e-02
  1.10329092e-02  4.29250896e-02 -5.78164645e-02  7.06526125e-03
  3.23381368e-03 -1.05536739e-04  1.10573322e-03  3.54209468e-02
 -1.78255897e-03 -1.64010376e-01  1.58311069e-01 -1.54311076e-01
  2.14787126e-02  6.33864384e-03  9.78644788e-02  6.31409092e-03
 -1.09129608e-01 -1.44032016e-01 -1.35497466e-01  1.70862570e-01
 -3.72074991e-02  3.55072762e-03 -2.78916936e-02  2.21769959e-02
  4.03649509e-02 -2.67130490e-02 -3.43142003e-02 -1.35340750e-01
  1.26529083e-01  1.12133613e-02 -1.68469781e-03 -5.07491268e-03
 -1.22812748e-01 -2.25020200e-02 -6.64350390e-02  3.53668742e-02
  1.48493024e-02  4.26239371e-02  1.97408516e-02  8.27600155e-03
  4.58796956e-02  1.04015386e-02  2.34234091e-02  4.38811742e-02
 -1.62990093e-0

Cos Similarity  tf.Tensor(0.31035912, shape=(), dtype=float32)
(128,)
(128,)
[-2.61688560e-01  1.64560810e-01 -1.67439565e-01  4.62928005e-02
 -3.11742560e-03  4.77611413e-03  1.59490258e-01  1.85802411e-02
  7.70396146e-04 -1.79694537e-02 -1.12498671e-01 -4.93357610e-03
 -6.63064374e-03  1.92436919e-01 -2.46775895e-02 -1.93276972e-01
 -1.80226956e-02 -1.37895480e-01  7.60409757e-02 -1.06557153e-01
  1.21189669e-01  1.07266471e-01  3.34613435e-02 -1.97791085e-02
 -5.24162780e-03  2.24165851e-03  1.52138807e-02 -3.63399461e-02
 -6.92367032e-02 -1.37230992e-01  2.34495401e-02  2.31575202e-02
 -1.58572532e-02 -1.49273826e-02 -1.83653697e-01 -5.64143322e-02
 -1.40450997e-02  1.84290603e-01 -8.27543190e-05 -4.70498353e-02
 -5.09027466e-02 -2.92129768e-03 -1.53879607e-02 -3.01553519e-03
  1.61566302e-01  1.30435079e-01 -1.21400625e-01 -8.20192508e-03
  1.34453017e-04  9.58686322e-03 -1.56278443e-03  3.10723158e-03
 -1.81661490e-02  1.46719418e-03 -7.88638368e-03  9.08394810e-03
  1.48775741e

(128,)
(128,)
[-8.92218947e-02 -1.54994158e-02  2.71937493e-02 -1.29777357e-01
 -2.40372103e-02 -1.17793204e-02 -1.71135515e-01 -5.15960306e-02
 -4.80553042e-03 -4.91701299e-03  3.02110296e-02  3.41427536e-03
 -4.26368788e-03 -7.85178691e-02  4.70868461e-02 -1.84702843e-01
 -7.77585618e-03 -1.70953855e-01  1.54676676e-01  1.05711527e-01
  1.36179738e-02  1.62025034e-01  2.14872509e-02  1.68036371e-01
  1.95153139e-03 -2.52214889e-03  5.23712933e-02 -8.54952447e-03
 -9.85450894e-02  9.46647674e-02 -1.63011968e-01 -4.74354066e-02
 -5.78602171e-03 -1.47599587e-02  1.94312662e-01 -1.01322811e-02
  1.07430657e-02 -1.17554061e-01 -1.33066485e-02 -4.89750924e-03
 -1.69518337e-01 -4.88290284e-03 -6.55345665e-03 -8.38978402e-03
 -6.91254362e-02 -1.24506250e-01 -6.22641966e-02  3.60176055e-04
 -3.77369724e-04 -5.93050336e-03  4.59689647e-03  1.31920958e-02
 -1.23700248e-02 -2.62984168e-03 -3.91050056e-03  1.52523052e-02
 -1.85309723e-01  9.42285266e-03 -5.70817618e-03  2.82881153e-03
 -4.7647208

(128,)
(128,)
[-0.049927    0.08942525  0.11564949  0.00670642  0.02783943  0.03034925
 -0.03597923  0.05755669  0.02415076 -0.00830173 -0.24291304 -0.0120011
  0.00197407  0.09455617  0.12557042 -0.11981091  0.02536472  0.22146
  0.10847419  0.02256006 -0.17021284  0.13366847  0.12128927 -0.1460407
  0.03707619 -0.00920484  0.03112279  0.00558541  0.16299869 -0.25852633
 -0.03400153  0.09661969 -0.04176549  0.01374596 -0.08950173  0.02069293
  0.01182431 -0.12378732 -0.02532608  0.17658487  0.19188106 -0.01312624
 -0.00291113 -0.01054572  0.05257926  0.03940377  0.15150301  0.00418959
  0.01416956  0.01594347  0.00724485  0.00505593 -0.00978027  0.00956502
  0.03251931  0.01185773  0.09830482 -0.00931522 -0.02819772 -0.00058973
  0.15274444 -0.18703386  0.01456397  0.00827017 -0.12984495  0.04698801
 -0.00278641  0.02198721  0.02356672 -0.08239224  0.05576288  0.18785402
  0.00555256  0.01400611  0.02574146  0.01999365  0.06952764 -0.18183243
 -0.01174696  0.03178226  0.00659857 -0.00

Cos Similarity  tf.Tensor(0.8793279, shape=(), dtype=float32)
(128,)
(128,)
[ 1.91688523e-01  3.99157666e-02 -8.48074704e-02  1.71579748e-01
 -2.62516867e-02  1.01582892e-02  1.94415748e-01 -6.89801993e-03
 -1.42523169e-03 -1.27629470e-03 -1.92671940e-01 -4.60281409e-03
  9.82097816e-03  1.87496454e-01 -6.93957806e-02  6.40635937e-02
 -8.03835038e-03 -5.44234142e-02  1.32770583e-01  1.27139717e-01
  8.51188414e-03 -1.53851017e-01  2.86512747e-02 -7.98773617e-02
  1.59278270e-02  5.12208510e-03  1.00474477e-01 -1.14595541e-03
 -7.04461476e-03 -4.91202474e-02  1.69587761e-01  1.70538828e-01
 -1.61862131e-02 -3.86917056e-03 -1.96080133e-01  2.89397314e-03
  2.06643972e-03 -4.40599397e-02 -5.21957409e-03 -1.08293280e-01
  7.87731409e-02  5.11144102e-03 -1.44685397e-03 -1.77740064e-02
  3.09589598e-02  4.76416051e-02  7.38202641e-03 -4.19521425e-03
 -2.47564283e-03 -6.26260880e-03 -4.50691814e-03  8.29648506e-03
 -1.55601185e-02  2.50977883e-03 -2.36362871e-03  1.17742838e-02
 -5.96284531e-

In [9]:
print(f'The accuracy of the model is: {accuracy}')

The accuracy of the model is: 0.7421875


This implementation with the `for` loop is great for understanding exactly what is going on when you are computing the accuracy, however you can implement this in a more efficient way using vectorization. You will do this in the assignment.

Feel free to add a new cell and try it out with this small dataset, and check that you are getting the same results. You can do this with exactly the same functions you used in the cell above.

**Congratulations on finishing this lecture notebook!** 

Now you should have a clearer understanding of how to evaluate your Siamese language models using the accuracy metric. 

**Keep it up!**